In [1]:
import pandas as pd
import os
import json



In [2]:
headers=["sentence_id", "sentence", "phrase_start", "phrase_end", 'phrase', 'num_native_annotator', "num_non_native_annotator", "num_complex_marks_native", "num_complex_marks_non_native", "complex_binary", "complex_prob"]

datasets=["../data/cwishareddataset/traindevset/english/Wikipedia_Dev.tsv","../data/cwishareddataset/traindevset/english/Wikipedia_Train.tsv", "../data/cwishareddataset/testset/english/Wikipedia_Test.tsv"]
dataset_names = ['Wikipedia_dev', "Wikipedia_train", "Wikipedia_test"]

data = pd.read_csv(datasets[0], sep='\t', names=headers)
data

,sentence_id,sentence,phrase_start,phrase_end,phrase,num_native_annotator,num_non_native_annotator,num_complex_marks_native,num_complex_marks_non_native,complex_binary,complex_prob
0,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,4,8,tail,10,10,0,2,1,0.10
1,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,12,25,Epidexipteryx,10,10,6,3,1,0.45
2,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,35,bore,10,10,0,3,1,0.15
3,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,53,bore unusual vertebrae,10,10,0,1,1,0.05
4,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,36,53,unusual vertebrae,10,10,2,0,1,0.10
...,...,...,...,...,...,...,...,...,...,...,...
689,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,100,109,knowledge,10,10,0,2,1,0.10
690,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,74,80,person,10,10,0,0,0,0.00
691,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,95,99,true,10,10,0,0,0,0.00
692,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,113,122,purifying,10,10,7,8,1,0.75


In [3]:
def count_words(row):
    phrase = row['phrase']
    return len(phrase.split())

def get_words(data):
    data["complex_prob_str"] = data.apply(lambda row: format(row['complex_prob'], ".2f"), axis=1 )
    data['num_words'] = data.apply(lambda row: count_words(row), axis=1)
    words = data[data['num_words']==1]
    return words
    
words = get_words(data)

In [4]:
words[['phrase', 'complex_prob', 'num_words']].sort_values(['complex_prob'], ascending=False)

,phrase,complex_prob,num_words
217,thermodynamics,1.00,1
427,unearthed,0.95,1
560,prototype,0.95,1
477,monasteries,0.95,1
261,enlargement,0.90,1
...,...,...,...
183,source,0.00,1
411,Birthday,0.00,1
410,Queen,0.00,1
409,Australia,0.00,1


In [15]:
def get_text_and_metadata(words):

    text = " ".join(words['phrase'].tolist())

    phrases = words['phrase'].to_list()
    weights = words['complex_prob'].to_list()
    

    word_weights = dict((phrase, weight) for phrase, weight in zip(phrases,weights))
    
    return text, word_weights


print(get_text_and_metadata(words))



("traded Toronto season Stewart Chicago early brought deal Bentley Max Leafs basis Russian language formed novelists style Turgenev Ivan Goncharov Ivan Leo subsequent Tolstoy lyric Lermontov poets Mikhail Vertical measurements distance referred direction commonly depth elevations However people living higher statistically significant rate higher suicide lack oxygen metres illnesses ft altitude sickness high altitude pulmonary edema high cerebral altitude edema altitude High defined begin metres ft sea level population census Quito populous city Ecuador Guayaquil Quito pronunciation Spanish formally ˈkito capital San Francisco Quito elevation city Ecuador feet meters sea official level highest capital city world responsibility countries physician mother midwife hospital administrator birth parents child properly registered appropriate agency government statue Accademia moved replaced Gallery Florence original replica location original version Costumbres recorded Selena hit Dulce album S

In [16]:
directory = "../data/evaluation/" 

if not os.path.exists(directory):
    os.makedirs(directory)

for name, dataset in zip(dataset_names, datasets):
    data = pd.read_csv(dataset, sep='\t', names=headers)
    words = get_words(data)
    text, wordset = get_text_and_metadata(words)

    
    
    with open(directory+name+".txt", "w")as f:
        f.write(text)
        
    with open(directory+name+".json", "w")as f:
        f.write(json.dumps(wordset))